In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset6/MORE-PLUS-DATASET/D_train.pkl
/kaggle/input/dataset6/MORE-PLUS-DATASET/O_train.pkl
/kaggle/input/dataset6/MORE-PLUS-DATASET/train_df.tsv
/kaggle/input/dataset6/MORE-PLUS-DATASET/D_val.pkl
/kaggle/input/dataset6/MORE-PLUS-DATASET/O_val.pkl
/kaggle/input/dataset6/MORE-PLUS-DATASET/val_df.tsv
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/684048166224838656.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/918577252350164992.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/828273258290110464.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/866154330222600192.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/694262101578227712.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/863638110311038976.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/721524447706877952.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/899682245799104512.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images/879814825471340546.jpg
/kaggle/input/dataset6/MORE-PLUS-DATASET/images

In [2]:
!pip install torch transformers pytorch_lightning evaluate pandas pillow
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9bca098ddf3cf947f54a038486a029bb3e960e5c3a713d968b35ae326d91eb21
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [4]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [5]:
# task3.py
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartModel, ViTModel, ViTImageProcessor
import pandas as pd
import pickle
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

import nltk
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('omw-1.4', download_dir='/kaggle/working/nltk_data')

# Point nltk to the custom download path
nltk.data.path.append('/kaggle/working/nltk_data')

from evaluate import load
from PIL import Image
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
PATH = '/kaggle/input/dataset6/MORE-PLUS-DATASET/'
MAX_LENGTH = 256
BART_MODEL = "facebook/bart-base"
VIT_MODEL = "google/vit-base-patch16-224"

class MOREPlusDataset(Dataset):
    def __init__(self, df_path, d_pkl, o_pkl, img_dir, tokenizer, img_processor):
        self.df = pd.read_csv(df_path, sep='\t')
        with open(d_pkl, 'rb') as f:
            self.descriptions = pickle.load(f)
        with open(o_pkl, 'rb') as f:
            self.objects = pickle.load(f)
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.img_processor = img_processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        pid = str(item['pid'])
        caption = item['text']
        desc = ' '.join(self.descriptions.get(pid, []))  # also handle missing descriptions safely

        objs_data = self.objects.get(pid, [])
        if isinstance(objs_data, dict):
            objs = ' '.join(list(objs_data.keys())[:36])
        elif isinstance(objs_data, list):
            objs = ' '.join(objs_data[:36])
        else:
            objs = ''

        target = item['target_of_sarcasm']
        explanation = item['explanation']

        input_seq = f"{caption} {desc} {objs} </s> {target}"
        inputs = self.tokenizer(input_seq, max_length=MAX_LENGTH,
                                padding='max_length', truncation=True,
                                return_tensors='pt')

        image_path = f"{self.img_dir}/{pid}.jpg"
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.img_processor(image, return_tensors='pt')['pixel_values'][0]
        except FileNotFoundError:
            image = torch.zeros((3, 224, 224))  # Handle missing image gracefully

        labels = self.tokenizer(explanation, max_length=MAX_LENGTH,
                                padding='max_length', truncation=True,
                                return_tensors='pt')['input_ids'][0]

        return {'input_ids': inputs['input_ids'][0],
                'attention_mask': inputs['attention_mask'][0],
                'image': image,
                'labels': labels}

class SharedFusion(nn.Module):
    def __init__(self, hidden_size=768):
        super().__init__()
        self.text_attn = nn.MultiheadAttention(hidden_size, 8, batch_first=True)
        self.image_attn = nn.MultiheadAttention(hidden_size, 8, batch_first=True)
        self.gate_v = nn.Linear(hidden_size, hidden_size)
        self.gate_t = nn.Linear(hidden_size, hidden_size)
        self.alpha1 = nn.Parameter(torch.tensor(0.25))
        self.alpha2 = nn.Parameter(torch.tensor(0.25))
        self.beta1 = nn.Parameter(torch.tensor(0.25))
        self.beta2 = nn.Parameter(torch.tensor(0.25))

    def forward(self, text_emb, image_emb):
        A_t, _ = self.text_attn(text_emb, text_emb, text_emb)
        A_v, _ = self.image_attn(image_emb, image_emb, image_emb)

        F_vt = A_t * image_emb
        F_tv = A_v * text_emb

        G_v = torch.sigmoid(self.gate_v(image_emb))
        G_t = torch.sigmoid(self.gate_t(text_emb))

        F1 = G_v * F_tv + (1-G_v) * F_vt
        F2 = G_t * F_tv + (1-G_t) * F_vt

        F_v = G_v * image_emb + (1-G_v) * F_tv
        F_t = G_t * text_emb + (1-G_t) * F_vt

        return self.alpha1*F1 + self.alpha2*F2 + self.beta1*F_v + self.beta2*F_t


class TURBO(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.tokenizer = BartTokenizer.from_pretrained(BART_MODEL)
        self.bart = BartModel.from_pretrained(BART_MODEL)
        self.vit = ViTModel.from_pretrained(VIT_MODEL)
        self.img_processor = ViTImageProcessor.from_pretrained(VIT_MODEL)
        self.fusion = SharedFusion()
        self.lm_head = nn.Linear(768, self.bart.config.vocab_size)
        self.rouge = load('rouge')
        self.bleu = load('bleu')
        self.meteor = load('meteor')
        self.bertscore = load('bertscore')
        self.validation_outputs = []

    def setup(self, stage=None):
        self.train_dataset = MOREPlusDataset(PATH+'train_df.tsv', PATH+'D_train.pkl', PATH+'O_train.pkl', PATH+'images', self.tokenizer, self.img_processor)
        self.val_dataset = MOREPlusDataset(PATH+'val_df.tsv', PATH+'D_val.pkl', PATH+'O_val.pkl', PATH+'images', self.tokenizer, self.img_processor)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=8, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=8)

    def validation_step(self, batch, batch_idx):
        logits = self(**batch)
        loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), batch['labels'].view(-1))
        
        preds = torch.argmax(logits, dim=-1)
        pred_text = self.tokenizer.batch_decode(preds, skip_special_tokens=True)
        true_text = self.tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
    
        # Explicitly log val_loss
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
    
        pred_text = [p if p.strip() else "empty prediction" for p in pred_text]
        true_text = [t if t.strip() else "empty reference" for t in true_text]
    
        self.validation_outputs.append({'predictions': pred_text, 'references': true_text})
    
        if batch_idx == 0:
            self.sample_predictions = list(zip(pred_text, true_text))
    
        return loss

    def on_validation_epoch_end(self):
        preds, targets = [], []
        for output in self.validation_outputs:
            preds.extend(output['predictions'])
            targets.extend(output['references'])
    
        filtered_preds_targets = [(p, t) for p, t in zip(preds, targets) if p.strip() and t.strip()]
        if not filtered_preds_targets:
            print("No valid predictions for metrics calculation.")
            return
    
        preds, targets = zip(*filtered_preds_targets)
    
        # Compute ROUGE, METEOR, BERTScore as usual
        rouge = self.rouge.compute(predictions=preds, references=targets)
        meteor = self.meteor.compute(predictions=preds, references=targets)
        bertscore = self.bertscore.compute(predictions=preds, references=targets, lang='en')
    
        # Compute BLEU-1 to BLEU-4 explicitly
        bleu1 = self.bleu.compute(predictions=preds, references=[[t] for t in targets], max_order=1)
        bleu2 = self.bleu.compute(predictions=preds, references=[[t] for t in targets], max_order=2)
        bleu3 = self.bleu.compute(predictions=preds, references=[[t] for t in targets], max_order=3)
        bleu4 = self.bleu.compute(predictions=preds, references=[[t] for t in targets], max_order=4)
    
        # Explicitly log metrics
        self.log_dict({
            'val_rouge1': rouge['rouge1'],
            'val_rouge2': rouge['rouge2'],
            'val_rougeL': rouge['rougeL'],
            'val_bleu1': bleu1['bleu'],
            'val_bleu2': bleu2['bleu'],
            'val_bleu3': bleu3['bleu'],
            'val_bleu4': bleu4['bleu'],
            'val_meteor': meteor['meteor'],
            'val_bertscore': torch.tensor(bertscore['f1']).mean(),
        }, prog_bar=True)
    
        # Clearly print a few generated examples
        print("\nSample Generated Sarcasm Explanations:")
        for idx, (pred, true) in enumerate(self.sample_predictions[:5]):
            print(f"\nExample {idx+1}:")
            print(f"Generated Explanation: {pred}")
            print(f"Actual Explanation: {true}")
    
        # Clear for next epoch
        self.validation_outputs.clear()


    def forward(self, input_ids, attention_mask, image, labels=None):
        text_emb = self.bart.encoder(input_ids, attention_mask).last_hidden_state
        img_emb = self.vit(image).last_hidden_state.mean(dim=1, keepdim=True).expand_as(text_emb)
        fused = self.fusion(text_emb, img_emb)
        out = self.bart.decoder(input_ids=labels, encoder_hidden_states=fused).last_hidden_state
        return self.lm_head(out)

    def training_step(self, batch, batch_idx):
        logits = self(**batch)
        loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), batch['labels'].view(-1))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)


# Trainer
if __name__ == '__main__':
    checkpoint_callback = ModelCheckpoint(
        dirpath='checkpoints/',
        filename='turbo-epoch{epoch:02d}-val_loss{val_loss:.4f}',
        save_top_k=-1,
        every_n_epochs=1,
        monitor='val_loss',
    )
    
    trainer = pl.Trainer(
        max_epochs=5, 
        accelerator='gpu',
        callbacks=[checkpoint_callback]
    )
    
    model = TURBO()
    trainer.fit(model)

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

...


Sample Generated Sarcasm Explanations:

Example 1:
Generated Explanation: the author is pissed at <user> for not getting network in malad.
Actual Explanation: the author is pissed at <user> for not getting network in malad.

Example 2:
Generated Explanation: people worst than waiting for an hour on the t wheel for a gate to come open in snowy, windy chicago.
Actual Explanation: nothing worst than waiting for an hour on the tarmac for a gate to come open in snowy, windy chicago.

Example 3:
Generated Explanation: nobody likes getting one hour of their life sob away.
Actual Explanation: nobody likes getting one hour of their life sucked away.

Example 4:
Generated Explanation: trump a plant foodary pet i reply on monday morning is not a good way to start the new week.
Actual Explanation: having a salivary gland biopsy on monday morning is not a good way to start the new week.

Example 5:
Generated Explanation: the author is worried that the weekend is going to be freezing with a high of

Validation: |          | 0/? [00:00<?, ?it/s]


Sample Generated Sarcasm Explanations:

Example 1:
Generated Explanation: the author is pissed at <user> for not getting network in malad.
Actual Explanation: the author is pissed at <user> for not getting network in malad.

Example 2:
Generated Explanation: to worst than waiting for an hour on the t sidewalks for a gate to come open in snowy, windy chicago.
Actual Explanation: nothing worst than waiting for an hour on the tarmac for a gate to come open in snowy, windy chicago.

Example 3:
Generated Explanation: thereody likes getting one hour of their life bunny away.
Actual Explanation: nobody likes getting one hour of their life sucked away.

Example 4:
Generated Explanation: when a colary volcanosis on monday morning is not a good way to start the new week.
Actual Explanation: having a salivary gland biopsy on monday morning is not a good way to start the new week.

Example 5:
Generated Explanation: the author is worried that the weekend is going to be freezing with a high of -1 a

Validation: |          | 0/? [00:00<?, ?it/s]


Sample Generated Sarcasm Explanations:

Example 1:
Generated Explanation: the author is pissed at <user> for not getting network in malad.
Actual Explanation: the author is pissed at <user> for not getting network in malad.

Example 2:
Generated Explanation: when worst than waiting for an hour on the t sidewalks for a gate to come open in snowy, windy chicago.
Actual Explanation: nothing worst than waiting for an hour on the tarmac for a gate to come open in snowy, windy chicago.

Example 3:
Generated Explanation: nobody likes getting one hour of their life bunny away.
Actual Explanation: nobody likes getting one hour of their life sucked away.

Example 4:
Generated Explanation: these a styl avarythuriosis on monday morning is not a good way to start the new week.
Actual Explanation: having a salivary gland biopsy on monday morning is not a good way to start the new week.

Example 5:
Generated Explanation: the author is worried that the weekend is going to be freezing with a high of -

In [6]:
!ls /kaggle/working/checkpoints

'turbo-epochepoch=00-val_lossval_loss=0.1532.ckpt'
'turbo-epochepoch=01-val_lossval_loss=0.1035.ckpt'
'turbo-epochepoch=02-val_lossval_loss=0.0869.ckpt'
'turbo-epochepoch=03-val_lossval_loss=0.0980.ckpt'
'turbo-epochepoch=04-val_lossval_loss=0.0821.ckpt'


In [7]:
metric=trainer.callback_metrics
print(metric)

{'train_loss': tensor(0.0094), 'val_loss': tensor(0.0821), 'val_rouge1': tensor(0.8714), 'val_rouge2': tensor(0.7951), 'val_rougeL': tensor(0.8713), 'val_bleu1': tensor(0.8826), 'val_bleu2': tensor(0.8415), 'val_bleu3': tensor(0.8045), 'val_bleu4': tensor(0.7699), 'val_meteor': tensor(0.8903), 'val_bertscore': tensor(0.9721)}


In [8]:
import tensorflow as tf

event_file = "/kaggle/working/lightning_logs/version_0/events.out.tfevents.1744129026.1e5ef3c85fca.31.0"

for event in tf.compat.v1.train.summary_iterator(event_file):
    for value in event.summary.value:
        print(f"Step: {event.step}, Tag: {value.tag}, Value: {value.simple_value}")

NotFoundError: /kaggle/working/lightning_logs/version_0/events.out.tfevents.1744129026.1e5ef3c85fca.31.0; No such file or directory